In [ ]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

MAIN_URL = "https://kli.korean.go.kr/term/indexMain.do?lang=kr"

DOMAIN = "Accounting"
DEPT_ID = "004"
OUTPUT_JSON = "kli_terms_426_3_all.json"

TOTAL_PAGES = 50      # 전체 페이지 수
BLOCK_SIZE = 5          # 1~5, 6~10, 11~15 ... 형식


def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    return driver


def open_search_result_list(driver):
    """메인 → lclas_name_426 → bubble_lst_426 li[3]/a 클릭 후 검색결과 리스트 진입"""
    driver.get(MAIN_URL)
    time.sleep(2)

    # 1. 대분류 클릭
    box = driver.find_element(By.ID, "lclas_name_425")
    box.find_element(By.TAG_NAME, "p").click()
    time.sleep(1)

    # 2. 버블 3번째 클릭
    driver.find_element(
        By.XPATH, '//*[@id="bubble_lst_425"]/li[6]/a'
    ).click()

    # 3. 리스트 페이지 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.url_contains("indexSearchList.do")
    )
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "includeSearchList"))
    )
    time.sleep(1)
    print("검색 결과 리스트 페이지 진입 완료:", driver.current_url)


def parse_current_page(driver):
    """현재 페이지에서 title / answer 추출"""
    results = []

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//*[@id='includeSearchList']//strong[@class='kor']")
            )
        )
    except TimeoutException:
        print("  - strong.kor 요소가 안 보임 → 0개 처리")
        return results

    items = driver.find_elements(
        By.XPATH,
        "//*[@id='includeSearchList']//a[.//strong[@class='kor']]"
    )
    print(f"  - 항목 수: {len(items)}")

    for idx, a in enumerate(items, start=1):
        try:
            title_el = a.find_element(By.XPATH, ".//strong[@class='kor']")
            title = title_el.text.strip()

            answer_el = a.find_element(By.XPATH, ".//p[@class='txt']")
            answer = answer_el.text.strip()

            if not title and not answer:
                continue

            results.append({
                "title": title,
                "answer": answer,
                "domain": DOMAIN,
                "dept_id": DEPT_ID
            })
        except NoSuchElementException:
            print(f"    · {idx}번째 항목: title/answer 없음 → skip")
        except Exception as e:
            print(f"    · {idx}번째 항목 예외: {e} → skip")

    return results


def click_page_number(driver, page_num):
    """
    현재 블럭 안에서 'page_num'에 해당하는 페이지 번호 a 태그 클릭.
    - class는 num / num current active
    - onclick은 searchInfo.setPage(page_num)
    """
    page_str = str(page_num)

    # 페이지 변경 감지를 위한 현재 첫 title
    try:
        first_title_before = driver.find_element(
            By.XPATH, "//*[@id='includeSearchList']//strong[@class='kor']"
        ).text
    except Exception:
        first_title_before = None

    # 후보 XPATH 리스트 (텍스트 / onclick 기반)
    candidates = [
        f"//*[@id='includeSearchList']//a[normalize-space(text())='{page_str}']",
        f"//*[@id='includeSearchList']//a[contains(@onclick,'setPage({page_num})')]",
        f"//a[contains(@onclick,'setPage({page_num})')]",
    ]

    link = None
    for xp in candidates:
        try:
            link = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, xp))
            )
            break
        except TimeoutException:
            continue

    if not link:
        print(f"  - 페이지 번호 {page_str} 링크를 찾지 못함")
        return False

    driver.execute_script("arguments[0].click();", link)

    # current active가 page_num이 될 때까지(or title 변경) 대기
    try:
        WebDriverWait(driver, 10).until(
            lambda d: (
                d.find_elements(
                    By.XPATH,
                    f"//*[@id='includeSearchList']//a[contains(@class,'current') and normalize-space(text())='{page_str}']"
                )
                or (
                    first_title_before is not None and
                    d.find_element(
                        By.XPATH, "//*[@id='includeSearchList']//strong[@class='kor']"
                    ).text != first_title_before
                )
            )
        )
    except Exception:
        time.sleep(1.5)

    return True


def click_next_block(driver):
    """
    '다음' 버튼 클릭 (5개 페이지 블럭 단위 이동)
    xpath 가 div[11]/div[12] 사이에서 바뀔 수 있으니 여러 후보 사용
    """
    candidate_xpaths = [
        "//*[@id='includeSearchList']/div[2]/div[12]/a[12]",
        "//*[@id='includeSearchList']/div[2]/div[11]/a[12]",
        "//*[@id='includeSearchList']//a[contains(text(),'다음')]",
    ]

    next_btn = None
    for xp in candidate_xpaths:
        try:
            next_btn = driver.find_element(By.XPATH, xp)
            break
        except NoSuchElementException:
            continue

    if not next_btn:
        print("  - '다음' 버튼을 찾지 못함 → 마지막 블럭으로 판단")
        return False

    driver.execute_script("arguments[0].click();", next_btn)
    time.sleep(1.5)
    return True


def crawl_block_pagination(total_pages=TOTAL_PAGES, block_size=BLOCK_SIZE, max_terms=None):
    """
    1~total_pages를 block_size(5) 단위로 크롤링.
    - 1~5 페이지 크롤링 → '다음' → 6~10 → '다음' → 11~15 → ...
    """
    driver = create_driver()
    open_search_result_list(driver)

    all_terms = []

    # 첫 블럭: 1 페이지는 이미 열려 있으므로 거기서 시작
    for block_start in range(1, total_pages + 1, block_size):
        block_end = min(block_start + block_size - 1, total_pages)
        print(f"\n===== 블럭 {block_start} ~ {block_end} 처리 =====")

        for page in range(block_start, block_end + 1):
            # 첫 블럭의 1페이지는 이미 열려 있으니 클릭 생략
            if not (block_start == 1 and page == 1):
                print(f"\n→ 페이지 {page} 이동 시도")
                moved = click_page_number(driver, page)
                if not moved:
                    print(f"페이지 {page} 이동 실패 → 여기서 종료")
                    driver.quit()
                    return all_terms

            print(f"===== 페이지 {page} 크롤링 =====")
            page_terms = parse_current_page(driver)
            print(f"  → {len(page_terms)}개 수집")
            all_terms.extend(page_terms)

            if max_terms is not None and len(all_terms) >= max_terms:
                print(f"\n수집 개수 {len(all_terms)}가 max_terms {max_terms} 도달 → 중단")
                driver.quit()
                return all_terms

        # 마지막 블럭이면 '다음' 누르지 않고 종료
        if block_end >= total_pages:
            print("\n모든 블럭 처리 완료")
            break

        print(f"\n블럭 {block_start} ~ {block_end} 완료 → '다음' 버튼으로 다음 블럭 이동")
        moved_block = click_next_block(driver)
        if not moved_block:
            print("다음 블럭으로 이동 실패 → 여기서 종료")
            break

    driver.quit()
    return all_terms

if __name__ == "__main__":
    terms = crawl_block_pagination(total_pages=TOTAL_PAGES, block_size=BLOCK_SIZE)
    print(f"\n총 수집 용어 수: {len(terms)}")
    
    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(terms, f, ensure_ascii=False, indent=2)
    print("저장 완료 →", OUTPUT_JSON)


검색 결과 리스트 페이지 진입 완료: https://kli.korean.go.kr/term/search/indexSearchList.do?lang=kr

===== 블럭 1 ~ 5 처리 =====
===== 페이지 1 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 2 이동 시도
===== 페이지 2 크롤링 =====
  - 항목 수: 10
    · 4번째 항목: title/answer 없음 → skip
    · 7번째 항목: title/answer 없음 → skip
  → 8개 수집

→ 페이지 3 이동 시도
===== 페이지 3 크롤링 =====
  - 항목 수: 10
    · 5번째 항목: title/answer 없음 → skip
    · 7번째 항목: title/answer 없음 → skip
    · 9번째 항목: title/answer 없음 → skip
  → 7개 수집

→ 페이지 4 이동 시도
===== 페이지 4 크롤링 =====
  - 항목 수: 10
    · 4번째 항목: title/answer 없음 → skip
    · 7번째 항목: title/answer 없음 → skip
  → 8개 수집

→ 페이지 5 이동 시도
===== 페이지 5 크롤링 =====
  - 항목 수: 10
    · 4번째 항목: title/answer 없음 → skip
  → 9개 수집

블럭 1 ~ 5 완료 → '다음' 버튼으로 다음 블럭 이동

===== 블럭 6 ~ 10 처리 =====

→ 페이지 6 이동 시도
===== 페이지 6 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 7 이동 시도
===== 페이지 7 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 8 이동 시도
===== 페이지 8 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페이지 9 이동 시도
===== 페이지 9 크롤링 =====
  - 항목 수: 10
  → 10개 수집

→ 페

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import json


BASE_LIST_URL = "https://taxlaw.nts.go.kr/st/USESTJ001P.do"

# 결과 저장 리스트
results = []


def create_driver():
    options = webdriver.ChromeOptions()
    # 필요 시 주석 해제
    # options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver


def wait_and_get_answer(driver, wait, timeout=10):
    """
    상세 페이지에서 ntstTrgyDictEpnCntn 의 텍스트를
    '비어 있지 않을 때'까지 기다렸다가 가져온다.
    """
    try:
        # 요소가 생길 때까지 1차 대기
        wait.until(
            EC.presence_of_element_located((By.ID, "ntstTrgyDictEpnCntn"))
        )

        # 텍스트가 비어있지 않을 때까지 2차 대기 (최대 timeout 초)
        def text_not_empty(d):
            try:
                el = d.find_element(By.ID, "ntstTrgyDictEpnCntn")
                txt = el.text.strip()
                if txt:
                    return True
                # .text가 비어있으면 innerText도 한 번 체크
                inner = el.get_attribute("innerText") or ""
                return inner.strip() != ""
            except Exception:
                return False

        WebDriverWait(driver, timeout).until(text_not_empty)

        el = driver.find_element(By.ID, "ntstTrgyDictEpnCntn")
        txt = el.text.strip()
        if not txt:
            txt = (el.get_attribute("innerText") or "").strip()
        return txt

    except TimeoutException:
        # 끝까지 기다렸는데도 비어 있으면 빈 문자열 리턴
        return ""


def crawl():
    driver = create_driver()
    wait = WebDriverWait(driver, 10)

    try:
        # 1. 목록 페이지 접속
        driver.get(BASE_LIST_URL)

        page_index = 1

        while True:
            print(f"\n=== {page_index} 페이지 시작 ===")

            # 2-1. 현재 페이지 용어 목록 로딩 대기
            wait.until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]')
                )
            )

            # 2-2. 현재 페이지의 모든 row 개수 확인
            rows = driver.find_elements(By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr')
            row_count = len(rows)
            print(f"행 개수: {row_count}")

            if row_count == 0:
                print("테이블에 행이 없습니다. 종료합니다.")
                break

            # 각 행 처리
            for index in range(row_count):
                # back() 이후 stale 방지: 매번 새로 rows 조회
                rows = driver.find_elements(By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr')

                if index >= len(rows):
                    break

                row = rows[index]
                title_td = row.find_element(By.XPATH, './td[1]')
                title_text = title_td.text.strip()

                print(f"[{page_index:02d}페이지, {index+1:02d}번째] 제목: {title_text}")

                # 상세 페이지로 이동
                try:
                    link = title_td.find_element(By.TAG_NAME, 'a')
                    driver.execute_script("arguments[0].scrollIntoView(true);", link)
                    time.sleep(0.2)
                    link.click()
                except NoSuchElementException:
                    driver.execute_script("arguments[0].scrollIntoView(true);", title_td)
                    time.sleep(0.2)
                    title_td.click()

                # 상세 페이지에서 answer 텍스트 가져오기
                answer_text = wait_and_get_answer(driver, wait, timeout=10)

                # JSON 객체 생성
                item = {
                    "title": title_text,
                    "answer": answer_text,
                    "domain": "Accounting",
                    "dept_id": "004",
                }
                results.append(item)

                # answer 앞부분 로깅 (디버그용)
                preview = (answer_text[:50] + "…") if len(answer_text) > 50 else answer_text
                print(f"  → 수집 완료 (answer 길이: {len(answer_text)}, 미리보기: {preview})")

                # 목록 페이지로 복귀
                driver.back()

                # 목록 재로딩 대기
                try:
                    wait.until(
                        EC.presence_of_element_located(
                            (By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]')
                        )
                    )
                except TimeoutException:
                    print("목록 페이지로 돌아오지 못했습니다. 크롤링을 종료합니다.")
                    return

            # 2-6. 다음 페이지로 이동
            print(f"{page_index} 페이지 처리 완료. 다음 페이지로 이동 시도.")

            try:
                # a.next 요소가 클릭 가능한 상태일 때까지 대기
                next_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "#paging .next"))
                )
            except TimeoutException:
                print("더 이상 '다음' 버튼이 없습니다. 마지막 페이지로 판단하고 종료합니다.")
                break

            # 다음 버튼이 비활성화된 경우(class로 구분 가능한 경우)
            try:
                cls = next_button.get_attribute("class") or ""
                if "disabled" in cls or "off" in cls:
                    print("'다음' 버튼이 비활성화 상태입니다. 종료합니다.")
                    break
            except StaleElementReferenceException:
                print("'다음' 버튼 참조가 사라졌습니다. 종료합니다.")
                break

            # 현재 첫 번째 행의 제목 저장 (다음 페이지 이동 여부 검증용)
            first_title_before = driver.find_element(
                By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]'
            ).text.strip()

            # 스크롤 후 JS로 클릭
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(0.3)
            driver.execute_script("arguments[0].click();", next_button)

            # 다음 페이지 로딩 대기
            time.sleep(1)
            try:
                wait.until(
                    EC.presence_of_element_located(
                        (By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]')
                    )
                )
            except TimeoutException:
                print("다음 페이지 로딩 실패. 종료합니다.")
                break

            first_title_after = driver.find_element(
                By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]'
            ).text.strip()

            if first_title_before == first_title_after:
                print("다음 페이지로 넘어가지 않았습니다. 마지막 페이지로 판단하고 종료합니다.")
                break

            page_index += 1

    finally:
        driver.quit()


if __name__ == "__main__":
    crawl()

    print(f"\n총 수집 개수: {len(results)}")
    with open("tax_terms.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print("tax_terms.json 파일로 저장 완료")



=== 1 페이지 시작 ===
행 개수: 10
[01페이지, 01번째] 제목: 1가구 1주택
  → 수집 완료 (answer 길이: 215, 미리보기: 1가구 1주택이라 함은 주민등록법에 의한 세대별 주민등록표에 기재되어 있는 세대주와 그 가…)
[01페이지, 02번째] 제목: 1세대
  → 수집 완료 (answer 길이: 322, 미리보기: 1세대란 거주자 및 그 배우자(법률상 이혼을 하였으나 생계를 같이 하는 등 사실상 이혼한 …)
[01페이지, 03번째] 제목: 1세대1주택
  → 수집 완료 (answer 길이: 481, 미리보기: 거주자의 1세대1주택과 그에 부수되는 토지를 양도함으로써 발생하는 소득에 대하여는 양도소득…)
[01페이지, 04번째] 제목: IFRS
  → 수집 완료 (answer 길이: 626, 미리보기: 기업의 회계처리와 재무제표에 대한 국제적 통일성을 높이기 위해 국제회계기준위원회에서 마련해…)
[01페이지, 05번째] 제목: 가결산
  → 수집 완료 (answer 길이: 239, 미리보기: 정규적인 회계기간 말에 행하는 본결산과 달리 회계기간 도중에 가마감으로 행하는 결산을 가결…)
[01페이지, 06번째] 제목: 가계정
  → 수집 완료 (answer 길이: 199, 미리보기: 거래가 발생하여 재산의 변화는 있었으나 귀속될 계정과목이 불명확하다든가, 계정과목은 확정되…)
[01페이지, 07번째] 제목: 가공이익
  → 수집 완료 (answer 길이: 147, 미리보기: 사실상의 이익이 아니라 회계장부상으로만 발생되는 이익을 말한다. 예를 들면 회사가 감가상각…)
[01페이지, 08번째] 제목: 가등기
  → 수집 완료 (answer 길이: 237, 미리보기: 본등기(本登記)의 순위(順位)보전을 위하여 하는 예비등기를 말한다. 가등기는 부동산물권·임…)
[01페이지, 09번째] 제목: 가사관련비
  → 수집 완료 (answer 길이: 240, 미리보기: 소득세법상의 개념으로 거주자가 가사와 관련하여 지출한 비용, 즉 개인적인 생계

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import time
import json


BASE_LIST_URL = "https://taxlaw.nts.go.kr/st/USESTJ001P.do"

START_PAGE = 100
OUTPUT_FILE = "tax_terms_from_100.json"

results = []


def create_driver():
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver


def wait_and_get_answer(driver, timeout=10):
    wait = WebDriverWait(driver, timeout)

    try:
        wait.until(
            EC.presence_of_element_located((By.ID, "ntstTrgyDictEpnCntn"))
        )

        def text_not_empty(d):
            try:
                el = d.find_element(By.ID, "ntstTrgyDictEpnCntn")
                txt = el.text.strip()
                if txt:
                    return True
                inner = el.get_attribute("innerText") or ""
                return inner.strip() != ""
            except Exception:
                return False

        WebDriverWait(driver, timeout).until(text_not_empty)

        el = driver.find_element(By.ID, "ntstTrgyDictEpnCntn")
        txt = el.text.strip()
        if not txt:
            txt = (el.get_attribute("innerText") or "").strip()
        return txt

    except TimeoutException:
        return ""


def crawl(start_page=1):
    driver = create_driver()
    wait = WebDriverWait(driver, 10)

    try:
        driver.get(BASE_LIST_URL)

        page_index = 1

        while True:
            # 목록 로딩 대기
            wait.until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]')
                )
            )

            # 현재 페이지 번호 로그
            print(f"\n=== {page_index} 페이지 시작 ===")

            # 아직 start_page 전에 있으면: 이 페이지는 건너뛰고 다음으로만 이동
            if page_index < start_page:
                print(f"{page_index} 페이지는 건너뜀 (start_page={start_page})")

                # '다음' 버튼 클릭 로직
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "#paging .next"))
                    )
                except TimeoutException:
                    print("더 이상 '다음' 버튼이 없습니다. 종료합니다.")
                    break

                first_title_before = driver.find_element(
                    By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]'
                ).text.strip()

                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(0.3)
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(1)

                try:
                    wait.until(
                        EC.presence_of_element_located(
                            (By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]')
                        )
                    )
                except TimeoutException:
                    print("다음 페이지 로딩 실패. 종료합니다.")
                    break

                first_title_after = driver.find_element(
                    By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]'
                ).text.strip()

                if first_title_before == first_title_after:
                    print("다음 페이지로 넘어가지 않았습니다. 마지막 페이지로 판단하고 종료합니다.")
                    break

                page_index += 1
                continue  # 여기서 rows 크롤링은 안 함

            # 여기부터는 page_index >= start_page 인 페이지들만 크롤링
            rows = driver.find_elements(By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr')
            row_count = len(rows)
            print(f"행 개수: {row_count}")

            if row_count == 0:
                print("테이블에 행이 없습니다. 종료합니다.")
                break

            for index in range(row_count):
                rows = driver.find_elements(By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr')
                if index >= len(rows):
                    break

                row = rows[index]
                title_td = row.find_element(By.XPATH, './td[1]')
                title_text = title_td.text.strip()

                print(f"[{page_index:02d}페이지, {index+1:02d}번째] 제목: {title_text}")

                try:
                    link = title_td.find_element(By.TAG_NAME, 'a')
                    driver.execute_script("arguments[0].scrollIntoView(true);", link)
                    time.sleep(0.2)
                    link.click()
                except NoSuchElementException:
                    driver.execute_script("arguments[0].scrollIntoView(true);", title_td)
                    time.sleep(0.2)
                    title_td.click()

                answer_text = wait_and_get_answer(driver, timeout=10)

                item = {
                    "title": title_text,
                    "answer": answer_text,
                    "domain": "Accounting",
                    "dept_id": "004",
                }
                results.append(item)

                preview = (answer_text[:50] + "…") if len(answer_text) > 50 else answer_text
                print(f"  → 수집 완료 (answer 길이: {len(answer_text)}, 미리보기: {preview})")

                driver.back()

                try:
                    wait.until(
                        EC.presence_of_element_located(
                            (By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]')
                        )
                    )
                except TimeoutException:
                    print("목록 페이지로 돌아오지 못했습니다. 크롤링을 종료합니다.")
                    return

            # 페이지 크롤링 후 다음 페이지로 이동
            print(f"{page_index} 페이지 처리 완료. 다음 페이지로 이동 시도.")

            try:
                next_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "#paging .next"))
                )
            except TimeoutException:
                print("더 이상 '다음' 버튼이 없습니다. 마지막 페이지로 판단하고 종료합니다.")
                break

            first_title_before = driver.find_element(
                By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]'
            ).text.strip()

            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            time.sleep(0.3)
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(1)

            try:
                wait.until(
                    EC.presence_of_element_located(
                        (By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]')
                    )
                )
            except TimeoutException:
                print("다음 페이지 로딩 실패. 종료합니다.")
                break

            first_title_after = driver.find_element(
                By.XPATH, '//*[@id="trgyDictAdmTable"]/tbody/tr[1]/td[1]'
            ).text.strip()

            if first_title_before == first_title_after:
                print("다음 페이지로 넘어가지 않았습니다. 마지막 페이지로 판단하고 종료합니다.")
                break

            page_index += 1

    finally:
        driver.quit()


if __name__ == "__main__":
    crawl(start_page=START_PAGE)

    print(f"\n총 수집 개수: {len(results)}")
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print(f"{OUTPUT_FILE} 파일로 저장 완료")



=== 1 페이지 시작 ===
1 페이지는 건너뜀 (start_page=100)

=== 2 페이지 시작 ===
2 페이지는 건너뜀 (start_page=100)

=== 3 페이지 시작 ===
3 페이지는 건너뜀 (start_page=100)

=== 4 페이지 시작 ===
4 페이지는 건너뜀 (start_page=100)

=== 5 페이지 시작 ===
5 페이지는 건너뜀 (start_page=100)

=== 6 페이지 시작 ===
6 페이지는 건너뜀 (start_page=100)

=== 7 페이지 시작 ===
7 페이지는 건너뜀 (start_page=100)

=== 8 페이지 시작 ===
8 페이지는 건너뜀 (start_page=100)

=== 9 페이지 시작 ===
9 페이지는 건너뜀 (start_page=100)

=== 10 페이지 시작 ===
10 페이지는 건너뜀 (start_page=100)

=== 11 페이지 시작 ===
11 페이지는 건너뜀 (start_page=100)

=== 12 페이지 시작 ===
12 페이지는 건너뜀 (start_page=100)

=== 13 페이지 시작 ===
13 페이지는 건너뜀 (start_page=100)

=== 14 페이지 시작 ===
14 페이지는 건너뜀 (start_page=100)

=== 15 페이지 시작 ===
15 페이지는 건너뜀 (start_page=100)

=== 16 페이지 시작 ===
16 페이지는 건너뜀 (start_page=100)

=== 17 페이지 시작 ===
17 페이지는 건너뜀 (start_page=100)

=== 18 페이지 시작 ===
18 페이지는 건너뜀 (start_page=100)

=== 19 페이지 시작 ===
19 페이지는 건너뜀 (start_page=100)

=== 20 페이지 시작 ===
20 페이지는 건너뜀 (start_page=100)

=== 21 페이지 시작 ===
21 페이지는 건너뜀 (start_page=100)

=== 22 페이